In [ ]:
from auth import generate_access_token
from consumer_details import DOMAIN
import requests
import pandas as pd

In [ ]:
access_token = generate_access_token()

In [ ]:
# access_token
headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json'
}

In [ ]:

# Set up the request headers

# Make the request
response = requests.get(DOMAIN+'/services/data/v48.0/sobjects/Account/001G100000IZAs5IAH', headers=headers)

# Extract the data from the response
data = response.json()

In [ ]:
def query(soql_query):
    try:
        endpoint = '/services/data/v48.0/query'
        records = []
        response = requests.get(DOMAIN + endpoint, headers=headers, params={'q': soql_query})
      
        total_size = response.json()['totalSize']
        records.extend(response.json()['records'])
        while not response.json()['done']:    
            response = requests.get(DOMAIN  + response.json()['nextRecordsUrl'],headers=headers)
            records.extend(response.json()['records'])
        return {'record_size': total_size, 'records': records}
    except Exception as e:
        print(e)
        return 


        

In [ ]:
# SOQL query
soql_query = "SELECT Id, Name FROM account"
results = query(soql_query)
print(results)
df = pd.DataFrame(results['records'])
df.head()

In [48]:
def retrive_ojbect_metadata(object_name):
    response = requests.get(DOMAIN + '/services/data/v56.0/sobjects/' + object_name + '/describe', headers=headers)
    return response.json()

In [50]:
response = retrive_ojbect_metadata('Account')

In [53]:
metadata = pd.DataFrame(response['fields'])

In [54]:
metadata

,aggregatable,aiPredictionField,autoNumber,byteLength,calculated,calculatedFormula,cascadeDelete,caseSensitive,compoundFieldName,controllerName,...,restrictedDelete,restrictedPicklist,scale,searchPrefilterable,soapType,sortable,type,unique,updateable,writeRequiresMasterRead
0,True,False,False,18,False,None,False,False,None,None,...,False,False,0,False,tns:ID,True,id,False,False,False
1,False,False,False,0,False,None,False,False,None,None,...,False,False,0,False,xsd:boolean,True,boolean,False,False,False
2,True,False,False,18,False,None,False,False,None,None,...,False,False,0,False,tns:ID,True,reference,False,False,False
3,True,False,False,765,False,None,False,False,Name,None,...,False,False,0,False,xsd:string,True,string,False,True,False
4,True,False,False,240,False,None,False,False,Name,None,...,False,False,0,False,xsd:string,True,string,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,False,False,False,0,False,None,False,False,None,None,...,False,False,0,False,xsd:boolean,True,boolean,False,True,False
314,True,False,False,765,False,None,False,False,None,None,...,False,True,0,False,xsd:string,True,picklist,False,True,False
315,False,False,False,4099,False,None,False,False,None,None,...,False,True,0,False,xsd:string,False,multipicklist,False,True,False
316,True,False,False,120,False,None,False,False,None,None,...,False,False,0,False,xsd:string,True,string,False,True,False


In [55]:
metadata.to_csv('metadata.csv')

In [103]:
def add_records(data):
    # /services/data/vXX.X/composite/sobjects
    response = requests.post(DOMAIN + '/services/data/v56.0/composite/sobjects/',json = data , headers=headers)
    return response.json()

def update_records(data):
    # /services/data/vXX.X/composite/sobjects
    response = requests.patch(DOMAIN + '/services/data/v56.0/composite/sobjects/',json = data, headers=headers)
    return response.json()

def upsert(data,object_name,ExternalId):
    SobjectName = object_name
    ExternalIdFieldName =ExternalId
    # /services/data/vXX.X/composite/sobjects
    response = requests.patch(DOMAIN + f'/services/data/v56.0/composite/sobjects/{SobjectName}/{ExternalIdFieldName}',json = data, headers=headers)
    return response.json()

In [101]:
request = {
   "allOrNone" : False,
   "records" : [{
      "attributes" : {"type" : "Account"},
      "Name" : "example.com",
      "BillingCity" : "San Francisco"
   }, {
      "attributes" : {"type" : "Contact"},
      "LastName" : "Johnson updated",
      'id' : '003G100000HihDxIAJ',
      "FirstName" : "Erica"
   }]
}

response = update_records(request)1
response


[{'success': False,
  'errors': [{'statusCode': 'MISSING_ARGUMENT',
    'message': 'Id not specified in an update call',
    'fields': []}]},
 {'id': '003G100000HihDxIAJ', 'success': True, 'errors': []}]

In [110]:
data = {
    "allOrNone" : False,
    "records" : [{
        "attributes" : {"type" : "Invester__c"},
        "Name" : "Company One updated",
        "ACCOUNT_NUMBER__c" : "174"
    }, {
        "attributes" : {"type" : "Invester__c"},
        "Name" : "Company new",
        "ACCOUNT_NUMBER__c" : "2000"
    }]
}

upsert(data,'Invester__c','ACCOUNT_NUMBER__c')

[{'id': 'a1fG1000000FuoHIAS', 'success': True, 'errors': [], 'created': False},
 {'id': 'a1fG1000000FurVIAS', 'success': True, 'errors': [], 'created': True}]